# 20 Questions

*Ryan Becwar and Matthew Dragan*

## Introduction
An intelligent agent should be able to make decicions given some information.  In addition, such an agent should be able to learn about something new that it hasn't seen before.  In this project we will try to produce a simplified version of the described agent.  The agent will play a version of 20 questions. In this version the goal objects will consist of fruits and vegetables, and the questions will be specialized for differentiating between fruits and vegetables.  This version of the game will determine the next question based on which question divides the remaining possiblilites the most evenly.  In a sense, the algorithm will do a binary search to find the most likely solution. Each question/answer combo has an associated weight, and each time a question is played and the right answer is guessed by the agent the weights will be updated.      

In [1]:
import numpy as np
import pandas as pd

In [5]:
class twentyQ(object):
    def __init__(self):
        self.questions = []
        self.answers = {}
        self.likelihood = {}
        self.weightVals = {}
        self.questionsUsed = []
        self.remainingFood = []
        
        data, weights = self.readData()
        
        self.processData(data, weights)
        
    def readData(self):
        # get known data from csv
        data = pd.read_csv('tempData.csv')

        # get weights from csv
        weights = pd.read_csv('tempWeights.csv')

        # extract questions
        questions = list(data.dtypes.index)
        self.questions = questions[1:]

        #extract data
        data = data.values

        # extract weights
        weights = weights.values
        
        return data, weights
    
    def processData(self, data, weights):
        for i in data:
            self.answers[i[0]] = i[1:]
            self.likelihood[i[0]] = 0
            self.remainingFood.append(i[0])
    
        for i in weights:
            self.weightVals[i[0]] = i[1:]
            
    def getFirstQuestion(self):
        countYes = 0
        countNo = 0
        nextQ = []
        for j in range(0,len(self.questions)):
            for i in self.answers:
                if self.answers[i][j] == 1:
                    countYes = countYes + 1
                elif self.answers[i][j] == -1:
                    countNo = countNo + 1
            nextQ.append(abs(countYes - countNo))
            countYes = countNo = 0
        self.questionsUsed.append(self.questions[np.argmin(nextQ)])
        return self.questions[np.argmin(nextQ)]
            
    def getNextQuestion(self, currentQ, currentA):
        couldBe = []
        countYes = 0
        countNo = 0
        nextQ = []
        for i in self.answers:
            if self.answers[i][self.questions.index(currentQ)] is currentA:
                couldBe.append(i)
        self.remainingFood = list(set(self.remainingFood) & set(couldBe))
        for j in range(0,len(self.questions)):
            for i in self.remainingFood:
                if self.answers[i][j] == 1:
                    countYes = countYes + 1
                elif self.answers[i][j] == -1:
                    countNo = countNo + 1
            nextQ.append(abs(countYes - countNo))
            countYes = countNo = 0
        self.questionsUsed.append(self.questions[np.argmin(nextQ)])
        return self.questions[np.argmin(nextQ)]
    
    def convertAnswer(self, currentA):
        if currentA is 'yes':
            return 1
        elif currentA is 'no':
            return -1
        else:
            return 0
        
    def updateLikelihood(self, currentQ, currentA):
        for i in self.answers:
            if self.answers[i][self.questions.index(currentQ)] is currentA:
                self.likelihood[i] = self.likelihood[i] + self.weightVals[i][self.questions.index(currentQ)] *1
            else:
                self.likelihood[i] = self.likelihood[i] + self.weightVals[i][self.questions.index(currentQ)] *-1
                
    def updateWeights(self, answer, correct):
        if correct is True:
            for i in self.questionsUsed:
                self.weightVals[answer][self.questions.index(i)] = self.weightVals[answer][self.questions.index(i)] + (1-self.weightVals[answer][self.questions.index(i)])/2 
                print(self.weightVals[answer][self.questions.index(i)])
        else:
            for i in self.questionsUsed:
                self.weightVals[answer][self.questions.index(i)] = self.weightVals[answer][self.questions.index(i)] - (1-self.weightVals[answer][self.questions.index(i)])/2
                if self.weightVals[answer][self.questions.index(i)] < .0625:
                    self.weightVals[answer][self.questions.index(i)] = .5
                    self.answers[answer][self.questions.index(i)] = -self.answers[answer][self.questions.index(i)]
    
    def writeToCSV(self):
        data = pd.read_csv('tempData.csv')

        # get weights from csv
        weights = pd.read_csv('tempWeights.csv')

In [9]:
me = twentyQ()

In [10]:
print(me.getFirstQuestion())
me.updateLikelihood(me.questionsUsed[0], -1)
print(me.getNextQuestion(me.questionsUsed[0], -1))
print(me.remainingFood)
me.updateLikelihood(me.questionsUsed[1],-1)
print(me.getNextQuestion(me.questionsUsed[1], -1))
print(me.remainingFood)
me.updateLikelihood(me.questionsUsed[2], 1)
print(me.getNextQuestion(me.questionsUsed[2], 1))
print(me.remainingFood)
me.updateWeights('raspberry', False)
me.likelihood


is it a fruit?
Is it commonly found on a sandwich?
['onion', 'lettuce', 'broccoli', 'carrot', 'potato']
Does it grow underground?
['broccoli', 'carrot', 'potato']
Is it usually round?
['carrot', 'potato']


{'apple': -0.5,
 'broccoli': 0.5,
 'carrot': 1.5,
 'lettuce': -0.5,
 'onion': 0.5,
 'orange': -0.5,
 'peach': -0.5,
 'potato': 1.5,
 'raspberry': -0.5,
 'strawberry': -0.5}

In [204]:
L1 = [1, 2, 3]
L2 = [3, 4, 5]

list(set(L1) & set(L2))

[3]

In [17]:
data = pd.read_csv('tempData.csv')
data['Unnamed: 0'][1]


'orange'